# Importing requirements

In [12]:
import os
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv
import json

In [13]:
# loading api and initalizing gpt

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

gpt_model= "gpt-4o-mini"

openai = OpenAI()

# Creating chat function

In [14]:
sys_msg= "You are a helpful assistant for an Airline called FlightAI.\
Give short, courteous answers, no more than 1 sentence. \
Always be accurate. If you don't know the answer, say so."

In [15]:
def chat(message, history):
    messages = [{"role":"system", "content": sys_msg}] + history + [{"role":"user", "content": message}]

    resp = openai.chat.completions.create(
        model = gpt_model,
        messages = messages,
        stream = True
    )

    response = ""
    for chunk in resp:
        response += chunk.choices[0].delta.content or ''
        yield response

# Creating Tools

In [16]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [17]:
#checking above function
get_ticket_price("london")

Tool get_ticket_price called for london


'$799'

In [18]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [19]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [20]:
def chat(message, history):
    messages = [{"role":"system", "content": sys_msg}] + history + [{"role":"user", "content": message}]
    resp = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)

    if resp.choices[0].finish_reason=="tool_calls":
        message = resp.choices[0].message
        resp, city = handle_tool_call(message)
        messages.append(message)
        messages.append(resp)
        resp = openai.chat.completions.create(model=gpt_model, messages=messages)
    
    return resp.choices[0].message.content

In [21]:

# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [22]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
